# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [1]:
!pip install torchsummaryX wandb --quiet

In [2]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm

import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Device:  cuda


In [3]:
!pip install speechpy
import speechpy


In [4]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',   'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [6]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"lambertbizimana","key":"9caf806b2f92ee446fb9a93edf5369d0"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73256 sha256=37b22a30e0cfb41e1c98d5f8f00b690004fa5bdcf7a7399e649c5750a0995836
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [7]:
# commands to download data from kaggle

!kaggle competitions download --force 11785-hw1p2-f23

!unzip -qo /kaggle/working/11785-hw1p2-f23.zip -d '/kaggle/working/'

100%|██████████████████████████████████████▉| 3.99G/3.99G [00:16<00:00, 228MB/s]
100%|███████████████████████████████████████| 3.99G/3.99G [00:16<00:00, 264MB/s]


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [8]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):


    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       =  f"{root}{partition}/mfcc/"# NotImplemented
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = f"{root}{partition}/transcript/"# NotImplemented

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = os.listdir(self.mfcc_dir) #NotImplemented
        mfcc_names.sort()
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    =  os.listdir(self.transcript_dir) #NotImplemented
        transcript_names.sort()

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)
        phonemes_n = 0
        for j in range(len(mfcc_names)):
            mfcc        = np.load(f"{self.mfcc_dir}{mfcc_names[j]}")
            phonemes_n += len(mfcc)

        self.mfccs, self.transcripts = np.zeros((phonemes_n,28),dtype=np.float32), np.zeros((phonemes_n),dtype=np.uint8)

        # TODO: Iterate through mfccs and transcripts
        self_mfccs_len = 0
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(f"{self.mfcc_dir}{mfcc_names[i]}") #NotImplemented
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = speechpy.processing.cmvn(mfcc)
        #   Load the corresponding transcript
            transcript  = np.load(f"{self.transcript_dir}{transcript_names[i]}") #NotImplemented Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
            transcript = transcript[1:-1]
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self_mfccs_len_0 = self_mfccs_len
            self_mfccs_len += len(mfcc)
            self.mfccs[self_mfccs_len_0:self_mfccs_len] = mfcc
            transcript = [self.phonemes.index(j) for j in transcript]
            self.transcripts[self_mfccs_len_0:self_mfccs_len] = transcript

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        # self.mfccs          = np.concatenate(self.mfccs)#NotImplemented

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        # self.transcripts    = np.concatenate(self.transcripts)#NotImplemented
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)
        print("question: is self.length == phonemes_n ??\n Answer: ",self.length == phonemes_n)
        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is.
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        # self.mfccs = np.append(np.zeros((self.context,27)),self.mfccs,axis=0) #NotImplemented # TODO
        # self.mfccs = np.append(self.mfccs,np.zeros((self.context,27)),axis=0)

        self.mfccs = np.pad(self.mfccs, ((self.context,self.context),(0,0)), 'constant', constant_values=(0))

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        # set_trace()
        ############### for j,val in enumerate(self.transcripts):
        ###############   self.transcripts[j] = self.phonemes.index(val)
        # self.transcripts = [self.phonemes.index(j) for j in self.transcripts]#NotImplemented
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind:ind+2*self.context+1]#NotImplemented
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten()#NotImplemented # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [9]:
class AudioTestDataset(torch.utils.data.Dataset):


    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       =  f"{root}{partition}/mfcc/"# NotImplemented

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = os.listdir(self.mfcc_dir) #NotImplemented
        mfcc_names.sort()
        phonemes_n = 0
        for j in range(len(mfcc_names)):
            mfcc        = np.load(f"{self.mfcc_dir}{mfcc_names[j]}")
            phonemes_n += len(mfcc)

        self.mfccs = np.zeros((phonemes_n,28),dtype=np.float32)

        # TODO: Iterate through mfccs and transcripts
        self_mfccs_len = 0
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc        = np.load(f"{self.mfcc_dir}{mfcc_names[i]}") #NotImplemented
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = speechpy.processing.cmvn(mfcc)

        #   Append each mfcc to self.mfcc
            self_mfccs_len_0 = self_mfccs_len
            self_mfccs_len += len(mfcc)
            self.mfccs[self_mfccs_len_0:self_mfccs_len] = mfcc

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        # self.mfccs          = np.concatenate(self.mfccs)#NotImplemented

        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)
        print("question: is self.length == phonemes_n ??\n Answer: ",self.length == phonemes_n)
        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is.
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        # self.mfccs = np.append(np.zeros((self.context,27)),self.mfccs,axis=0) #NotImplemented # TODO
        # self.mfccs = np.append(self.mfccs,np.zeros((self.context,27)),axis=0)

        self.mfccs = np.pad(self.mfccs, ((self.context,self.context),(0,0)), 'constant', constant_values=(0))


    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind:ind+2*self.context+1]#NotImplemented
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        frames = frames.flatten()#NotImplemented # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors

        return frames


# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [10]:
# config = {
#     'epochs'        :25,
#     'batch_size'    : 1024,
#     'context'       : 20,
#     'init_lr'       : 1e-4,
#     'architecture'  : 'very-low-cutoff',
#     'dataset_root'  : '/kaggle/working/11-785-f23-hw1p2/'
#     # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
# }

In [48]:
config = {
    'epochs'        :20,
    'batch_size'    : 8192,
    'context'       : 35,
    'init_lr'       : 1e-3,
    'architecture'  : 'double cylinder',
    'dropout'       : 0.2,
    'step_size'     : 1,
    'gamma'         : 0.05,
    'weight_decay'  : 0,
    'T_max'         : 10,
    'eta_min'       : 0,
    'factor'        : 0.5,
    'dataset_root'  : "/kaggle/working/11-785-f23-hw1p2/"
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [12]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(config["dataset_root"],
                          phonemes = PHONEMES,
                          context=config["context"],
                          partition= "train-clean-100")#NotImplemented

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(config["dataset_root"],
                          phonemes = PHONEMES,
                          context=config["context"],
                          partition= "dev-clean")#NotImplemented
# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(config["dataset_root"],
                          phonemes = PHONEMES,
                          context=config["context"],
                          partition= "test-clean")

question: is self.length == phonemes_n ??
 Answer:  True
question: is self.length == phonemes_n ??
 Answer:  True
question: is self.length == phonemes_n ??
 Answer:  True


In [49]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  8192
Context        :  35
Input size     :  1988
Output symbols :  42
Train dataset samples = 36091157, batches = 4406
Validation dataset samples = 1928204, batches = 236
Test dataset samples = 1934138, batches = 237


In [50]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([8192, 1988]) torch.Size([8192])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [51]:
# # This architecture will make you cross the very low cutoff
# # However, you need to run a lot of experiments to cross the medium or high cutoff
# class Network(torch.nn.Module):

#     def __init__(self, input_size, output_size):

#         super(Network, self).__init__()

#         self.model = torch.nn.Sequential(torch.nn.Linear(input_size, 2048),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.GELU(),
#                                  torch.nn.BatchNorm1d(2048),
#                                  torch.nn.Linear(2048, 2048),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.GELU(),
#                                  torch.nn.BatchNorm1d(2048),
#                                  torch.nn.Linear(2048, 2048),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.GELU(),
#                                  torch.nn.BatchNorm1d(2048),
#                                  torch.nn.Linear(2048, 1024),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.GELU(),
#                                  torch.nn.BatchNorm1d(1024),
#                                  torch.nn.Linear(1024, 1024),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.GELU(),
#                                  torch.nn.BatchNorm1d(1024),
#                                  torch.nn.Linear(1024, 1024),
#                                  torch.nn.GELU(),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.BatchNorm1d(1024),
#                                  torch.nn.Linear(1024, 1024),
#                                  torch.nn.GELU(),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.BatchNorm1d(1024),
#                                  torch.nn.Linear(1024, 512),
#                                  torch.nn.GELU(),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.BatchNorm1d(512),
#                                  torch.nn.Linear(512, 512),
#                                  torch.nn.GELU(),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.BatchNorm1d(512),
#                                  torch.nn.Linear(512, 512),
#                                  torch.nn.GELU(),
#                                  #torch.nn.GELU(inplace=True),
#                                  torch.nn.BatchNorm1d(512),
#                                  torch.nn.Linear(512, output_size))

#     def forward(self, x):
#       out = self.model(x)
#       return out


In [52]:
# class Network(torch.nn.Module):
#     def __init__(self, input_size, output_size):
#         super(Network, self).__init__()
        
#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, 2048),
#             torch.nn.Dropout(0.2),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.GELU(),
            
#             torch.nn.Linear(2048, 2048),
#             torch.nn.Dropout(0.2),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.GELU(),
            
#             torch.nn.Linear(2048, 2048),
#             torch.nn.Dropout(0.2),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.GELU(),
            
#             torch.nn.Linear(2048, 2048),
#             torch.nn.Dropout(0.2),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.GELU(),
            
#             torch.nn.Linear(2048, 2048),
#             torch.nn.Dropout(0.2),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.GELU(),
            
#             torch.nn.Linear(2048, 2048),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.GELU(),
            
#             torch.nn.Linear(2048, output_size)
#         )
        
#     def forward(self, x):
#         out = self.model(x)
#         return out
        

In [53]:
# # Best of mine
# #This architecture will make you cross the very low cutoff
# # However, you need to run a lot of experiments to cross the medium or high cutoff
# class Network(torch.nn.Module):
#     def __init__(self, input_size, output_size):
#         super(Network, self).__init__()

#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, 2048),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(2048, 2048),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(2048, 2048),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(2048),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(2048, 1024),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(1024),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(1024, 1024),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(1024),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(1024, 1024),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(1024),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(1024, 512),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(512),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(512, 512),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(512),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.5
#             torch.nn.Linear(512, 512),
#             torch.nn.GELU(),
#             torch.nn.BatchNorm1d(512),
#             torch.nn.Dropout(0.25),  # Add dropout with a probability of 0.25
#             torch.nn.Linear(512, output_size)
#         )

#     def forward(self, x):
#         out = self.model(x)
#         return out


In [54]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()
         # self.model.name = "zipper 2000"
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(1450,2000),
            torch.nn.BatchNorm1d(2000),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(2000,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(1450,1450),
            torch.nn.BatchNorm1d(1450),
            torch.nn.GELU(),
            torch.nn.Dropout(p = config['dropout']),

            torch.nn.Linear(1450, output_size) #Change to 40,out if using the previous layer
        )


    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [55]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
#summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 25 million parameters for HW1 (including ensembles)

In [56]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(model)

22485792

In [57]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

In [58]:
import os

# Define the checkpoint folder path
checkpoint_folder = "./checkpoints/"

# Create the folder if it doesn't exist
os.makedirs(checkpoint_folder, exist_ok=True)


In [59]:
checkpoint = torch.load('/kaggle/input/peaceg/model_epoch_5(1).pt')
model.load_state_dict(checkpoint)

<All keys matched successfully>

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [60]:
torch.cuda.empty_cache()
gc.collect()

579

In [61]:
from torch.nn.utils import clip_grad_norm_
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagationz

        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()
        # Clip gradients to prevent explosion
        #clip_grad_norm_(self.model.parameters(), max_norm=1.0)
        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [62]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

In [63]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
#from google.colab import drive
#drive.mount('/content/drive')

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [64]:
wandb.login(key="c1d931bb4beb9786d38d65463356ea4f554817f9") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [65]:
# Create your wandb run
run = wandb.init(
    name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

lr,▁
train_acc,▁
train_loss,▁
val_acc,▁
valid_loss,▁
lr,0.001
train_acc,88.99867
train_loss,0.29943
val_acc,86.52294
valid_loss,0.42413


In [66]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/kaggle/working/wandb/run-20230923_114202-o1391h0w/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

best_accuracy = 0


for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})
    actual_checkpoint = os.path.join(checkpoint_folder, f"model_epoch_{epoch +1}.pt")
    
    if val_acc >=best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), actual_checkpoint)
    
        #save on wandb
        wandb.save(actual_checkpoint)
#run.finish()


Epoch 1/20


Train:   0%|          | 0/4406 [00:00<?, ?it/s]

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 88.9923%	Train Loss 0.2996	 Learning Rate 0.0010000
	Val Acc 86.5052%	Val Loss 0.4242

Epoch 2/20


Train:   0%|          | 0/4406 [00:00<?, ?it/s]

...

	Train Acc 89.2536%	Train Loss 0.2914	 Learning Rate 0.0010000
	Val Acc 86.6449%	Val Loss 0.4220

Epoch 14/20


Train:   0%|          | 0/4406 [00:00<?, ?it/s]

Val:   0%|          | 0/236 [00:00<?, ?it/s]

	Train Acc 89.2612%	Train Loss 0.2909	 Learning Rate 0.0010000
	Val Acc 86.6501%	Val Loss 0.4230

Epoch 15/20


Train:   0%|          | 0/4406 [00:00<?, ?it/s]

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim= 1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions += [PHONEMES[j] for j in predicted_phonemes]# TODO

            ### Release memory
            del mfccs, predicted_phonemes, logits
            torch.cuda.empty_cache()

    return test_predictions

In [ ]:
predictions = test(model, test_loader)

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f23 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

In [ ]:
wasedfghjklsdfghjk,rtdyfygrtdhtfjygjhtf

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f23 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

In [ ]:
#############################Pretraining the model#######################################

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define your model architecture (make sure it matches the original model)
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model_eval       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
# summary(model, frames.to(device))
# Load the saved model state dictionary
model_eval.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/modelepoc3.pth"))


In [ ]:
config = {
    'epochs'        : 1,
    'batch_size'    : 1024,
    'context'       : 20,
    'init_lr'       : 1e-2,
    'architecture'  : 'very-low-cutoff',
    'dataset_root'  : "/content/data/11-785-f23-hw1p2/"
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model_eval.parameters(), lr= config['init_lr']) #Defining Optimizer

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
# wandb.watch(model, log="all")

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model_eval, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model_eval, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)

    torch.save(model_eval.state_dict(),"/content/drive/MyDrive/Colab Notebooks/premodelepoc3.pth")

In [ ]:
predictions = test(model_eval, test_loader)

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f23 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle